In [ ]:
import findspark
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.mllib import recommendation
from pyspark.mllib.recommendation import *

sc = SparkContext()

In [ ]:
''' define variables'''

rawUserArtistData = sc.textFile("file:///home/santho/Audio_recomm/data/user_artist_data.txt")
rawArtistData = sc.textFile("file:///home/santho/Audio_recomm/data/artist_data.txt")
rawArtistAlias = sc.textFile("file:///home/santho/Audio_recomm/data/artist_alias.txt")

In [ ]:
rawArtistData.first()

In [ ]:
def pairsplit(singlePair):
    splitPair = singlePair.rsplit('\t')
    if len(splitPair)!=2:
        return []
    else:
        try:
            return [(int(splitPair[0]), splitPair[1])]
        except:
            return []
artistByID = dict(rawArtistData.flatMap(lambda x: pairsplit(x)).collect())

In [ ]:
def aliaslookup(alias):
    splitPair = alias.rsplit('\t')
    if len(splitPair) !=2:
        return []
    else:
        try:
            return [(int(splitPair[0]),int(splitPair[1]))]
        except:
            return []
artistAlias = rawArtistAlias.flatMap(lambda x: aliaslookup(x)).collectAsMap()

In [ ]:
bArtistAlias = sc.broadcast(artistAlias)

In [ ]:
#Build a model

def ratinglookup(x):
    userID, artistID, count = map(lambda line: int(line),x.split())
    finalArtistID = bArtistAlias.value.get(artistID)
    if finalArtistID is None:
        finalArtistID = artistID
    return Rating(userID, finalArtistID, count)

trainData = rawUserArtistData.map(lambda x: ratinglookup(x))
trainData.cache()

In [ ]:
model = ALS.trainImplicit(trainData,10, 5)

In [ ]:
'''test artist'''

spotCheckingID = 2093760
bArtistByID = sc.broadcast(artistByID)

rawArtistsForUser = (trainData
                    .filter(lambda x: x.user == spotCheckingID)
                    .map(lambda x: bArtistByID.value.get(x.product))
                    .collect())
print(rawArtistsForUser)

In [ ]:
'''output Recommendation'''
recommendations = map(lambda x: artistByID.get(x.product),
                    model.call("recommendProducts", spotcheckingID, 10))
print(recommendations)